**Importing Libraries**

In [97]:
import pandas as pd
import numpy as np
import chainladder as cl
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.0f}'.format
import calendar
#pd.options.display.max_rows = 999

**Loading and Exploring data**

In [98]:
#premiums_df = pd.read_excel(r'C:\Users\Muhammad.Ismail\Downloads\UPR Motor Q2 2024.xlsm', sheet_name = 'Data')

In [99]:
premiums_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680563 entries, 0 to 680562
Data columns (total 66 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   Serial No.            680539 non-null  float64       
 1   Prd_Period            680538 non-null  object        
 2   Issue Date            460364 non-null  datetime64[ns]
 3   Risk Location         219697 non-null  object        
 4   Location Street       41383 non-null   object        
 5   Location City         41383 non-null   object        
 6   Location Building     40870 non-null   object        
 7   Line Of Business      680538 non-null  object        
 8   Division              680538 non-null  object        
 9   Department            412288 non-null  object        
 10  Business Type         680538 non-null  object        
 11  Sales Channel         680538 non-null  object        
 12  UW Year               680538 non-null  float64       
 13 

In [100]:
premiums_raw_df = premiums_df.iloc[:, 0:48]

In [101]:
premiums_raw_df

,Serial No.,Prd_Period,Issue Date,Risk Location,Location Street,Location City,Location Building,Line Of Business,Division,Department,Business Type,Sales Channel,UW Year,Product,Policy No.,Endorsement No.,Customer Type,Insured Name,Sum Insured,Inception date,Expiry Date,Gross Prem,Commission,Charge,Net Premium,RI Premium,QS Amount,Retention,SurPlus Amount,FAC Amount,WAR FAC Amount,WAR Treaty,WAR Spcl Retn,FAC Comm,QS SI,RETN SI,SUR SI,FAC SI,Spcl Retn,Diff,Transc Dt,Unnamed: 41,Inception<Expiry,Unnamed: 43,Expiry +1,TPL/Comprehensive,Premium - Net of CHG,Premium - Net of RI
0,1,2013,NaT,DUBAI,,,,M,Shk. Zayed Branch,NaN,Direct Business,Broker,"2,013",IMC,14,126696,Individual,SHARIF SAFY ELDIN HAMOUDAH,"1,536,000",2013-03-17,2014-04-17,"1,175",176,0,"1,175",NaN,0,"1,175",0,0,NaN,NaN,NaN,0,0,1536000,0,0,NaN,NaN,NaT,NaN,1,NaN,2014-04-18,Comprehensive,"1,175","1,175"
1,2,2013,NaT,DUBAI,,,,M,Shk. Zayed Branch,NaN,Direct Business,Broker,"2,013",IMC,28,125921,Individual,AZNITA BINT AZIZ,"1,535,000",2013-03-05,2014-04-05,"1,475",221,0,"1,475",NaN,0,"1,475",0,0,NaN,NaN,NaN,0,0,1535000,0,0,NaN,NaN,NaT,NaN,1,NaN,2014-04-06,Comprehensive,"1,475","1,475"
2,3,2013,NaT,,NaN,NaN,NaN,M,Shk. Zayed Branch,NaN,Direct Business,Broker,"2,013",IMC,53,123346,Corporate,Timna Electronics & Telecommunication (FZE),"1,260,000",2013-01-23,2014-02-23,"1,040",156,0,"1,040",NaN,0,"1,040",0,0,NaN,NaN,NaN,0,0,1260000,0,0,NaN,NaN,NaT,NaN,1,NaN,2014-02-24,Comprehensive,"1,040","1,040"
3,4,2013,NaT,,NaN,NaN,NaN,M,Shk. Zayed Branch,NaN,Direct Business,Broker,"2,013",IMC,56,126390,Corporate,JUMAIRA LIGHT INDUSTRIES,"1,944,000",2013-03-12,2014-04-12,"1,535",230,0,"1,535",NaN,0,"1,535",0,0,NaN,NaN,NaN,0,0,1944000,0,0,NaN,NaN,NaT,NaN,1,NaN,2014-04-13,Comprehensive,"1,535","1,535"
4,5,2013,NaT,,NaN,NaN,NaN,M,Shk. Zayed Branch,NaN,Direct Business,Broker,"2,013",IMC,107,122346,Individual,RAJESH BHANDARI,"1,575,000",2013-01-09,2014-02-09,"1,535",230,0,"1,535",NaN,0,"1,535",0,0,NaN,NaN,NaN,0,0,1575000,0,0,NaN,NaN,NaT,NaN,1,NaN,2014-02-10,Comprehensive,"1,535","1,535"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680558,"674,306",2024 Q2,2024-06-29,NaN,NaN,NaN,NaN,MOTOR,HEAD-OFFICE,MOTOR,DIRECT BUSUINESS,AGENT,"2,024",MOTOR COMPREHENSIVE-ETRADE,P/01/IMC/2024/00505,NaN,INDIVIDUAL,DANA BASSAM TOUMA,"103,277,000",2024-06-29,2025-07-28,"4,350",326,0,"4,350","4,350",0,"4,350",0,0,0,0,0,0,0,103277000,0,0,0,NaN,2024-06-29,NaN,1,NaN,2025-07-29,Comprehensive,"4,350","4,350"
680559,"674,307",2024 Q2,2024-06-28,NaN,NaN,NaN,NaN,MOTOR,ABU DHABI BRANCH,MOTOR,DIRECT BUSUINESS,AGENT,"2,024",MOTOR THIRD-PARTY LIABILITY-ETRADE,P/04/IMT/2024/00102,NaN,INDIVIDUAL,KHOURSHID ALI GUL,0,2024-06-29,2025-07-28,750,56,0,750,750,0,750,0,0,0,0,0,0,0,0,0,0,0,NaN,2024-06-29,NaN,1,NaN,2025-07-29,TPL,750,750
680560,"674,308",2024 Q2,2024-06-30,NaN,NaN,NaN,NaN,MOTOR,ABU DHABI BRANCH,MOTOR,DIRECT BUSUINESS,AGENT,"2,024",MOTOR COMPREHENSIVE-ETRADE,P/04/IMC/2024/00323,NaN,INDIVIDUAL,MOHD ESSA M AL ABBADI,"101,102,200",2024-06-30,2025-07-31,"1,800",72,0,"1,800","1,800",0,"1,800",0,0,0,0,0,0,0,101102200,0,0,0,NaN,2024-06-30,NaN,1,NaN,2025-08-01,Comprehensive,"1,800","1,800"
680561,"674,309",2024 Q2,2024-06-30,NaN,NaN,NaN,NaN,MOTOR,AJMAN,MOTOR,DIRECT BUSUINESS,Call Centre,"2,024",MOTOR COMP SCHEME,P/11/MCS/2024/00527,NaN,INDIVIDUAL,AHMED SALEH H. S. ALMUKHATTAN,"103,517,000",2024-06-30,2025-07-31,"1,170",29,0,"1,170","1,170",0,"1,170",0,0,0,0,0,0,0,103517000,0,0,0,NaN,2024-06-30,NaN,1,NaN,2025-08-01,Comprehensive,"1,170","1,170"


In [102]:
def calculate_proportion(start_date_series, end_date_series, current_date):
    start_date_series = pd.to_datetime(start_date_series)
    end_date_series = pd.to_datetime(end_date_series)
    current_date = pd.to_datetime(current_date)

    proportion = 1.0
    total_duration = (end_date_series - start_date_series).dt.days
    time_elapsed = (end_date_series - current_date).dt.days
    proportion = time_elapsed / total_duration
    proportion = 1 - proportion
    proportion = proportion.clip(lower=0, upper=1)
    
    return proportion

def apply_proportion_to_premiums(df, start_col, end_col, current_date, premium_col):
    df['Proportion'] = calculate_proportion(df[start_col], df[end_col], current_date)
    
    df['Adjusted Premiums'] = df['Proportion'] * df[premium_col]
    
    return df

current_date = '2024-06-30'
premiums_raw_df = apply_proportion_to_premiums(premiums_raw_df, 'Inception date', 'Expiry Date', current_date, 'Premium - Net of CHG')

In [103]:
(premiums_raw_df['Adjusted Premiums'] == premiums_df['GEP']).value_counts()

True     680538
False        25
Name: count, dtype: int64

In [104]:
(premiums_raw_df['Adjusted Premiums'] == premiums_df['GEP'])

0         True
1         True
2         True
3         True
4         True
          ... 
680558    True
680559    True
680560    True
680561    True
680562    True
Length: 680563, dtype: bool

In [105]:
difference = (premiums_raw_df['Adjusted Premiums'] - premiums_df['GEP']).abs()

max_difference_index = difference.idxmax()

row_with_max_difference = premiums_raw_df.loc[max_difference_index]

print(f"Row index with greatest difference: {max_difference_index}")
print(f"Row with greatest difference:\n{row_with_max_difference}")


Row index with greatest difference: 0
Row with greatest difference:
Serial No.                                       1
Prd_Period                                    2013
Issue Date                                     NaT
Risk Location                                DUBAI
Location Street                                   
Location City                                     
Location Building                                 
Line Of Business                                 M
Division                         Shk. Zayed Branch
Department                                     NaN
Business Type                      Direct Business
Sales Channel                               Broker
UW Year                                      2,013
Product                                        IMC
Policy No.                                      14
Endorsement No.                             126696
Customer Type                           Individual
Insured Name            SHARIF SAFY ELDIN HAMOUDAH
Sum Insured   

In [106]:
false_rows = premiums_raw_df[~(premiums_raw_df['Adjusted Premiums'] == premiums_df['GEP'])]
false_rows

,Serial No.,Prd_Period,Issue Date,Risk Location,Location Street,Location City,Location Building,Line Of Business,Division,Department,Business Type,Sales Channel,UW Year,Product,Policy No.,Endorsement No.,Customer Type,Insured Name,Sum Insured,Inception date,Expiry Date,Gross Prem,Commission,Charge,Net Premium,RI Premium,QS Amount,Retention,SurPlus Amount,FAC Amount,WAR FAC Amount,WAR Treaty,WAR Spcl Retn,FAC Comm,QS SI,RETN SI,SUR SI,FAC SI,Spcl Retn,Diff,Transc Dt,Unnamed: 41,Inception<Expiry,Unnamed: 43,Expiry +1,TPL/Comprehensive,Premium - Net of CHG,Premium - Net of RI,Proportion,Adjusted Premiums
251727,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
258668,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
278041,"278,040",NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
290012,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
303345,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
315618,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
328057,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
339758,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
352875,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
361975,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [107]:
premiums_raw_df['Adjusted Premiums'].sum()

769403941.6807321

**Filtering for TPL**

In [108]:
premiums_raw_df['TPL EP'] = 0
premiums_raw_df['TPL EP'] = premiums_raw_df.apply(
    lambda x: x['Adjusted Premiums'] if x['TPL/Comprehensive'] == 'TPL' else 0, axis=1)


In [109]:
premiums_raw_df['TPL GP'] = 0
premiums_raw_df['TPL GP'] = premiums_raw_df.apply(
    lambda x: x['Premium - Net of CHG'] if x['TPL/Comprehensive'] == 'TPL' else 0, axis=1)


In [110]:
premiums_raw_df['TPL EP'].sum()

115517962.84249538

**Dataframe Creation**

In [119]:
start_date = datetime(2017, 7, 1)
end_date = datetime(2027, 6, 1)

date = []
current_date = start_date

while current_date <= end_date:
    last_day = calendar.monthrange(current_date.year, current_date.month)[1]
    date_string = "{}-{}-{}".format(last_day, current_date.strftime('%b'), current_date.strftime('%y'))
    date.append([current_date.strftime('1-%b-%y'), date_string])

    current_date += relativedelta(months=1)


temp_df = pd.DataFrame(date, columns=['From', 'To'])

temp_df['From'] = pd.to_datetime(temp_df['From'], format='%d-%b-%y')
temp_df['To'] = pd.to_datetime(temp_df['To'], format='%d-%b-%y')

**Gross Premiums (GP)**

In [112]:
temp_df['GP'] = 0
for temp_index, temp_row in temp_df.iterrows():
    from_date = temp_row['From']
    to_date = temp_row['To']
    matching_premiums = premiums_raw_df[
        (premiums_raw_df['Inception date']  <= to_date)& ( premiums_raw_df['Inception date']  >= from_date)]
    premium_sum = matching_premiums['TPL GP'].sum()
    temp_df.at[temp_index, 'GP'] = premium_sum

C:\Users\Muhammad.Ismail\AppData\Local\Temp\ipykernel_14860\355145241.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '991487.02' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  temp_df.at[temp_index, 'GP'] = premium_sum


In [113]:
temp_df 

,From,To,GP
0,2017-07-01,2017-07-31,"991,487"
1,2017-08-01,2017-08-31,"945,268"
2,2017-09-01,2017-09-30,"738,114"
3,2017-10-01,2017-10-31,"1,110,282"
4,2017-11-01,2017-11-30,"1,054,467"
5,2017-12-01,2017-12-31,"1,316,802"
6,2018-01-01,2018-01-31,"928,302"
7,2018-02-01,2018-02-28,"782,414"
8,2018-03-01,2018-03-31,"993,394"
9,2018-04-01,2018-04-30,"1,533,689"


**Gross EP as at (Premium Sum)**

In [114]:
def calculate_proportional_premiums_vectorized(premiums_column, inception_column, expiry_column, to_dates_column, premiums_raw_df):
    """
    Vectorized function to calculate proportional premiums for each 'To' date row.
    
    If expiry date > 'To' date: Calculate the proportion of elapsed time between inception and 'To' date.
    Else if expiry date <= 'To' date: Add the full premium (i.e., multiply by 1).

    Parameters:
    - premiums_column: The column containing the premiums values.
    - inception_column: The column containing inception dates.
    - expiry_column: The column containing expiry dates.
    - to_dates_column: The column of 'To' dates from the smaller DataFrame.
    - premiums_raw_df: The DataFrame containing premium, inception, and expiry data.

    Returns:
    - A pandas Series containing the sum of proportional premiums for each 'To' date row.
    """
    
    # Ensure all date columns are in datetime format
    premiums_raw_df[inception_column] = pd.to_datetime(premiums_raw_df[inception_column])
    premiums_raw_df[expiry_column] = pd.to_datetime(premiums_raw_df[expiry_column])
    to_dates_column = pd.to_datetime(to_dates_column)
    
    # Calculate the total duration between inception and expiry for the entire dataset
    total_duration = (premiums_raw_df[expiry_column] - premiums_raw_df[inception_column]).dt.days
    
    # Initialize an empty list to store results
    results = []
    
    # Loop through the 'To' dates
    for to_date in to_dates_column:
        # Apply the condition for Expiry Date > 'To' Date
        
        print(to_date) # Printing the code to check how long it takes for each row. Vectorizing definitely made it faster.
        # You can comment the above print statement.
        
        valid_rows = premiums_raw_df
        
        # Calculate the elapsed duration between inception date and 'To' date for valid rows
        elapsed_duration = (to_date - valid_rows[inception_column]).dt.days
        
        # Calculate the proportion for rows where expiry date > 'To' date
        proportion = np.clip(elapsed_duration / total_duration, 0, 1)
        
        # Check where expiry date <= 'To' date and set proportion to 1 for those rows
        proportion[valid_rows[expiry_column] <= to_date] = 1
        
        # Multiply the proportion by the premiums
        proportional_premiums = proportion * valid_rows[premiums_column]
        
        # Sum the proportional premiums for this 'To' date and append to the results list
        results.append(proportional_premiums.sum())
    
    # Return the result as a pandas Series to maintain the index alignment with 'To' dates
    return pd.Series(results, index=to_dates_column.index)

temp_df['Premium Sum'] = calculate_proportional_premiums_vectorized('TPL GP', 'Inception date', 'Expiry Date', temp_df['To'], premiums_raw_df)


2017-07-31 00:00:00
2017-08-31 00:00:00
2017-09-30 00:00:00
2017-10-31 00:00:00
2017-11-30 00:00:00
2017-12-31 00:00:00
2018-01-31 00:00:00
2018-02-28 00:00:00
2018-03-31 00:00:00
2018-04-30 00:00:00
2018-05-31 00:00:00
2018-06-30 00:00:00
2018-07-31 00:00:00
2018-08-31 00:00:00
2018-09-30 00:00:00
2018-10-31 00:00:00
2018-11-30 00:00:00
2018-12-31 00:00:00
2019-01-31 00:00:00
2019-02-28 00:00:00
2019-03-31 00:00:00
2019-04-30 00:00:00
2019-05-31 00:00:00
2019-06-30 00:00:00
2019-07-31 00:00:00
2019-08-31 00:00:00
2019-09-30 00:00:00
2019-10-31 00:00:00
2019-11-30 00:00:00
2019-12-31 00:00:00
2020-01-31 00:00:00
2020-02-29 00:00:00
2020-03-31 00:00:00
2020-04-30 00:00:00
2020-05-31 00:00:00
2020-06-30 00:00:00
2020-07-31 00:00:00
2020-08-31 00:00:00
2020-09-30 00:00:00
2020-10-31 00:00:00
2020-11-30 00:00:00
2020-12-31 00:00:00
2021-01-31 00:00:00
2021-02-28 00:00:00
2021-03-31 00:00:00
2021-04-30 00:00:00
2021-05-31 00:00:00
2021-06-30 00:00:00
2021-07-31 00:00:00
2021-08-31 00:00:00


In [115]:
temp_df

,From,To,GP,Premium Sum
0,2017-07-01,2017-07-31,"991,487","17,150,571"
1,2017-08-01,2017-08-31,"945,268","17,844,026"
2,2017-09-01,2017-09-30,"738,114","18,546,695"
3,2017-10-01,2017-10-31,"1,110,282","19,319,975"
4,2017-11-01,2017-11-30,"1,054,467","20,128,951"
5,2017-12-01,2017-12-31,"1,316,802","21,025,622"
6,2018-01-01,2018-01-31,"928,302","21,975,629"
7,2018-02-01,2018-02-28,"782,414","22,858,354"
8,2018-03-01,2018-03-31,"993,394","23,847,306"
9,2018-04-01,2018-04-30,"1,533,689","24,831,719"


**Renaming and Reordering + getting Earned Premiums**

In [116]:
temp_df['Earned Premiums'] = 0 
temp_df.loc[1:, 'Earned Premiums'] = temp_df['Premium Sum'].diff().fillna(0).iloc[1:] 
temp_df = temp_df.rename(columns={'Premium Sum': 'Gross EP As at'})
temp_df = temp_df.rename(columns={'GP': 'Gross Premiums'})

C:\Users\Muhammad.Ismail\AppData\Local\Temp\ipykernel_14860\1306668965.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[6.93455068e+05 7.02668928e+05 7.73280000e+05 8.08975442e+05
 8.96671567e+05 9.50006927e+05 8.82724414e+05 9.88952075e+05
 9.84413100e+05 1.06217073e+06 1.06463470e+06 1.17058554e+06
 1.21521015e+06 1.20404179e+06 1.29016956e+06 1.33654100e+06
 1.45808567e+06 1.72459263e+06 1.68515857e+06 1.92444841e+06
 1.93205786e+06 2.07142819e+06 2.06211272e+06 2.21192369e+06
 2.25484628e+06 2.21390628e+06 2.33439146e+06 2.28020665e+06
 2.33743541e+06 2.27850795e+06 2.08109842e+06 2.12475964e+06
 2.04316536e+06 2.07405410e+06 1.96384918e+06 1.94129702e+06
 1.85101242e+06 1.72405278e+06 1.70831906e+06 1.58495232e+06
 1.55687292e+06 1.42396294e+06 1.22299531e+06 1.28695601e+06
 1.20878890e+06 1.21368255e+06 1.16960055e+06 1.16634362e+06
 1.14826468e+06 1.09808101e+06 1.11998896e+06 1.07462300e+06
 1.08337578e

**Final Output**

In [117]:
temp_df

,From,To,Gross Premiums,Gross EP As at,Earned Premiums
0,2017-07-01,2017-07-31,"991,487","17,150,571",0
1,2017-08-01,2017-08-31,"945,268","17,844,026","693,455"
2,2017-09-01,2017-09-30,"738,114","18,546,695","702,669"
3,2017-10-01,2017-10-31,"1,110,282","19,319,975","773,280"
4,2017-11-01,2017-11-30,"1,054,467","20,128,951","808,975"
5,2017-12-01,2017-12-31,"1,316,802","21,025,622","896,672"
6,2018-01-01,2018-01-31,"928,302","21,975,629","950,007"
7,2018-02-01,2018-02-28,"782,414","22,858,354","882,724"
8,2018-03-01,2018-03-31,"993,394","23,847,306","988,952"
9,2018-04-01,2018-04-30,"1,533,689","24,831,719","984,413"
